In [2]:
!pip install python-dotenv requests openai-agents


  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
Using cached pydantic-2.12.4-py3-none-any.whl (463 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.1 MB/s eta 0:00:00a 0:00:01
Using cached uvicorn-0.38.0-py3-none-any.whl (68 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.1━━━━━━━━━━━━━  3/10 [pydantic-core]
    Uninstalling pydantic_core-2.41.1:━━━━━━━━━━━━━━━━━━━━━━━━  3/10 [pydantic-core]
      Successfully uninstalled pydantic_core-2.41.1━━━━━━━━━━━  3/10 [pydantic-core]
  Attempting uninstall: pydantic╺━━━━━━━━━━━━━━━━━━━  5/10 [starlette]
    Found existing installation: pydantic 2.12.00m━━━━━━━━━━━  7/10 [pydantic]
    Uninstalling pydantic-2.12.0:╺━━━━━━━━━━━  7/10 [pydantic]
      Successfully uninstalled pydantic-2.12.0━━━━━━━━━━━  7/10 [pydantic]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [openai-agents]0m [openai-agents]


In [19]:
import os
import requests
from dotenv import load_dotenv

from agents import Agent, trace, Runner, ModelSettings
load_dotenv()


True

In [20]:
def brave_search(query: str, count: int = 5):
    api_key = os.getenv("BRAVE_API_KEY")
    if not api_key:
        raise ValueError("BRAVE_API_KEY is missing")

    url = "https://api.search.brave.com/res/v1/web/search"

    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": api_key,
    }

    params = {
        "q": query,
        "count": count,
    }

    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code != 200:
        raise RuntimeError(f"Brave API Error: {resp.text}")

    return resp.json()


In [21]:
def brave_search_tool(input: dict):
    return brave_search(
        query=input["query"],
        count=input.get("count", 5)
    )

brave_tool = {
    "type": "function",
    "name": "BraveWebSearch",
    "description": "Searches the web using Brave Search API.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {"type": "string"},
            "count": {"type": "number"}
        },
        "required": ["query"]
    },
    "function": brave_search_tool
}


In [22]:
INSTRUCTIONS = """
You are a research assistant.
Use BraveWebSearch whenever external information is needed.
Return structured, factual summaries.
"""

research_agent = Agent(
    name="BraveResearcher",
    instructions=INSTRUCTIONS,
    tools=[brave_tool],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="auto"),
)


In [ ]:
INSTRUCTIONS = """
You are a research assistant.
Use BraveWebSearch whenever external info is required.
Return concise, factual summaries.
"""

research_agent = Agent(
    name="BraveResearcher",
    instructions=INSTRUCTIONS,
    tools=custom_tools,     # <-- custom Brave tool
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="auto"),
)


In [16]:
message = "Latest AI Agents frameworks in 2025. Compare them."

with trace("Brave Search Agent"):
    result = await Runner.run(research_agent, message)


AttributeError: 'dict' object has no attribute 'name'